<a href="https://colab.research.google.com/github/allxdxd/Busqueda-de-datos-abiertos-Colombia/blob/main/datos_abiertos_web_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install playwright
!playwright install
!playwright install firefox

Playwright Host validation warning: 
╔══════════════════════════════════════════════════════╗
║ Host system is missing dependencies to run browsers. ║
║ Missing libraries:                                   ║
║     libwoff2dec.so.1.0.2                             ║
║     libgstgl-1.0.so.0                                ║
║     libgstcodecparsers-1.0.so.0                      ║
║     libharfbuzz-icu.so.0                             ║
║     libenchant-2.so.2                                ║
║     libsecret-1.so.0                                 ║
║     libhyphen.so.0                                   ║
║     libmanette-0.2.so.0                              ║
╚══════════════════════════════════════════════════════╝
    at validateDependenciesLinux (/usr/local/lib/python3.10/dist-packages/playwright/driver/package/lib/server/registry/dependencies.js:216:9)
    at process.processTicksAndRejections (node:internal/process/task_queues:95:5)
    at async Registry._validateHostRequirements (/usr/

In [76]:
from playwright.async_api import async_playwright
import asyncio
import numpy as np
import pandas as pd

funtion_to_get_Data = """(()=>{
    let cointainer_sup = [... document.querySelectorAll('.browse2-result-content.clearfix')].map(cointainer => {
            let titles = cointainer.childNodes[1].childNodes[1].childNodes[1].childNodes[1].innerText;

    let categories = cointainer.childNodes[1].childNodes[1].childNodes[1].childNodes[3]?.innerText;

    let descriptions = cointainer.childNodes[3].childNodes[1]?.innerText;

    let date = cointainer.childNodes[7].childNodes[1].childNodes[3].innerText;

    let views = cointainer.childNodes[7].childNodes[3].childNodes[3]?.innerText

    let links = cointainer.childNodes[1].childNodes[1].childNodes[1].childNodes[1].childNodes[1].href;

        return {
            titles,
            categories,
            descriptions,
            date,
            views,
            links
        };

    });

    return cointainer_sup;
})()"""

meses = {
    'enero': '01',
    'febrero': '02',
    'marzo': '03',
    'abril': '04',
    'mayo': '05',
    'junio': '06',
    'julio': '07',
    'agosto': '08',
    'septiembre': '09',
    'octubre': '10',
    'noviembre': '11',
    'diciembre': '12'
}

async def searchData(query, final = 2):
    query = query.replace(' ', '+')
    data = []
    async with async_playwright() as p:
        browser = await p.chromium.launch()
        for i in range(1,final):
          page = await browser.new_page()
          await page.goto(f"https://www.datos.gov.co/browse?limitTo=datasets&q={query}&sortBy=relevance&page={i}")
          temp = await page.evaluate(funtion_to_get_Data)
          await page.close()
          data.extend(temp)
        await browser.close()
        # Asignar valores nulos
        df = pd.DataFrame(data)
        df = df.replace({'none': np.NaN})
        # Convertir la columna 'date' a formato datetime sin usar localización
        def reemplazar_fecha(fecha):
            for mes, num in meses.items():
                fecha = fecha.replace(f'de {mes} de', f'/{num}/')
            return fecha.replace(' ', '')
        df['date'] = df['date'].apply(reemplazar_fecha)
        df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y')
        # Cambiar views a int
        df['views'] = df['views'].str.replace(' ', '').str.replace(',', '').astype(int)
        # Separar codigos SODA
        df['code'] = df['links'].str.split('/').str[-1]
        # Crear endpoint
        df['api'] = 'https://www.datos.gov.co/resource/' + df['code'] + '.json'
        return df

data = await searchData('camara de comercio', final=40)

In [77]:
data

,titles,categories,descriptions,date,views,links,code,api
0,EMPRESAS ENCUESTA TIC 2017,"Ciencia, Tecnología e Innovación",La Primera Gran Encuesta TIC articula esfuerzo...,2024-04-20,7744,https://www.datos.gov.co/Ciencia-Tecnolog%C3%A...,vgv5-wvxj,https://www.datos.gov.co/resource/vgv5-wvxj.json
1,EMPRESAS BIC,Estadísticas Nacionales,EMPRESAS BIC DE LA JURISDICCION DE LA CAMARA D...,2024-04-21,1841,https://www.datos.gov.co/Estad%C3%ADsticas-Nac...,6m4s-ym5n,https://www.datos.gov.co/resource/6m4s-ym5n.json
2,Empresas Mypymes,"Comercio, Industria y Turismo",Empresas Mypymes por sector,2024-04-21,1330,https://www.datos.gov.co/Comercio-Industria-y-...,a88e-5ew3,https://www.datos.gov.co/resource/a88e-5ew3.json
3,EMPRESAS ACTIVAS LEGALMENTE CONSTITUIDAS,"Comercio, Industria y Turismo",EMPRESAS ACTIVAS LEGALMENTE CONSTITUIDAS,2024-04-21,2361,https://www.datos.gov.co/Comercio-Industria-y-...,7tuu-ghn4,https://www.datos.gov.co/resource/7tuu-ghn4.json
4,Empresas zona industrial Cota,"Comercio, Industria y Turismo",Empresas de la zona industrial del municipio d...,2024-03-26,2841,https://www.datos.gov.co/Comercio-Industria-y-...,asm3-gfyn,https://www.datos.gov.co/resource/asm3-gfyn.json
...,...,...,...,...,...,...,...,...
385,Indice de Información Clasificada y Reservada,Función Pública,La Empresa de Servicios Públicos E.S.P. del mu...,2024-04-01,77,https://www.datos.gov.co/Funci%C3%B3n-p%C3%BAb...,iqab-nfi7,https://www.datos.gov.co/resource/iqab-nfi7.json
386,EDUA - Contratos de Arrendamiento Centro Comer...,"Vivienda, Ciudad y Territorio",EDUA - Contratos de Arrendamiento Centro Comer...,2024-04-20,415,https://www.datos.gov.co/Vivienda-Ciudad-y-Ter...,p3m3-zezq,https://www.datos.gov.co/resource/p3m3-zezq.json
387,Registros de activos de información 2021,None,Los activos de información son los datos cread...,2024-04-01,127,https://www.datos.gov.co/dataset/Registros-de-...,tbuc-vztw,https://www.datos.gov.co/resource/tbuc-vztw.json
388,Esquema de publicación ESSA - 2020,Minas y Energía,Esquema de publicación ESSA ley 1712.,2024-04-01,640,https://www.datos.gov.co/Minas-y-Energ%C3%ADa/...,gwpu-zzi5,https://www.datos.gov.co/resource/gwpu-zzi5.json
